In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 get_column_descriptions	 
load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 re	 save_dataframes	 
store_objects	 sys	 urllib	 



# Scraping the Technology Stats

In [2]:

from bs4 import BeautifulSoup

wiki_url = 'https://ageofempires.fandom.com'
def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = BeautifulSoup(page_html, 'html.parser')
    
    return page_soup

In [3]:

def get_content_text_div(page_url):
    page_soup = get_page_soup(page_url)
    content_text_selector = '#mw-content-text'
    content_text_div = page_soup.select(content_text_selector)[0]
    
    return content_text_div

In [4]:

def get_main_technology_page(content_text_div, main_df, unique_df, link_dict):
    for heading_span in content_text_div.select('span[class="mw-headline"]'):
        heading = heading_span.string.strip()
        print('{}: {}'.format(heading_span.parent.name, heading))
        siblings_list = list(heading_span.parent.next_siblings)
        for sibling in siblings_list:
            if sibling.name == 'h3':
                
                # You've gone too far
                break
            
            elif sibling.name == 'dl':
                page_url = '{}{}'.format(wiki_url, sibling.find_all('a')[0]['href'])
                unique_content_text_div = get_content_text_div(page_url)
                unique_df, link_dict = get_unique_technology_page(unique_content_text_div,
                                                                  unique_df, link_dict)
                break
            elif sibling.name == 'table':
                df = pd.read_html(str(sibling), header=0)[0]
                df['Heading'] = heading
                main_df = pd.concat([main_df, df], sort=True)
                print(df.shape)
                for tr in sibling.find_all('tr')[1:]:
                    link = tr.find_all('td')[0].find_all('a')[1]
                    link_title = link['title']
                    link_dict[link_title] = link['href']
                break
    
    return main_df, unique_df, link_dict

In [5]:

def get_unique_technology_page(content_text_div, unique_df, link_dict):
    for heading_span in content_text_div.select('h3 span[class="mw-headline"]'):
        heading = heading_span.string.strip()
        print('{}: {}'.format(heading_span.parent.name, heading))
        siblings_list = list(heading_span.parent.next_siblings)
        for sibling in siblings_list:
            if sibling.name == 'table':
                df = pd.read_html(str(sibling), header=0)[0]
                df['Heading'] = heading
                unique_df = pd.concat([unique_df, df], sort=True)
                print(df.shape)
                for tr in sibling.find_all('tr')[1:]:
                    link = tr.find_all('td')[1].find_all('a')[0]
                    link_title = link['title']
                    link_dict[link_title] = link['href']
                break
    
    return unique_df, link_dict

In [143]:

main_page_url = '{}/wiki/Technology_(Age_of_Empires_II)'.format(wiki_url)
main_content_text_div = get_content_text_div(main_page_url)
main_columns_list = ['Heading', 'Technology', 'Age', 'Building', 'Cost', 'Effect']
main_df = pd.DataFrame([], columns=main_columns_list)
unique_columns_list = ['Heading', 'Civilization', 'Unique technology', 'Effect']
unique_df = pd.DataFrame([], columns=unique_columns_list)
main_df, unique_df, link_dict = get_main_technology_page(main_content_text_div,
                                                         main_df, unique_df, {})

h2: Building technologies
(6, 6)
h2: Economy technologies
(18, 6)
h2: Monk technologies
(10, 6)
h2: Military technologies
h3: Infantry
(9, 6)
h3: Missile / siege
(14, 6)
h3: Cavalry
(8, 6)
h2: Naval technologies
(3, 6)
h2: Unique technologies
h3: Castle Age
(31, 4)
h3: Imperial Age
(31, 4)
h2: Miscellaneous technologies
(8, 6)


In [145]:

unique_df[unique_columns_list].sample(5).T

,30,19,29,26,24
Heading,Castle Age,Imperial Age,Imperial Age,Imperial Age,Castle Age
Civilization,Vikings,Malians,Vietnamese,Spanish,Saracens
Unique technology,Chieftains,Farimba,Paper Money,Supremacy,Madrasah
Effect,Infantry does more damage to cavalry,Attack of Stable units increased,Player and allies receive 500 gold,Villager combat skill increased,33% of a Monk's gold cost are returned if he dies


In [6]:

def get_conq_technology_page(content_text_div, main_df, link_dict):
    for heading_span in content_text_div.select('span[class="mw-headline"]'):
        heading = heading_span.string.strip()
        if heading == 'Technologies':
            print('{}: {}'.format(heading_span.parent.name, heading))
            siblings_list = list(heading_span.parent.next_siblings)
            for sibling in siblings_list:
                if sibling.name == 'table':
                    df = pd.read_html(str(sibling), header=0)[0]
                    df['Heading'] = heading
                    main_df = pd.concat([main_df, df], sort=True)
                    print(df.shape)
                    for tr in sibling.find_all('tr')[1:]:
                        link = tr.find_all('td')[0].find_all('a')[0]
                        link_title = link['title']
                        link_dict[link_title] = link['href']
                    break
    
    return main_df, link_dict

In [157]:

conq_page_url = '{}/wiki/Age_of_Empires_II:_The_Conquerors'.format(wiki_url)
conq_content_text_div = get_content_text_div(conq_page_url)
main_df, link_dict = get_conq_technology_page(conq_content_text_div, main_df,
                                              link_dict)

h3: Technologies
(7, 6)


In [302]:

main_df = main_df.drop_duplicates(subset=['Age', 'Building', 'Cost'], keep='first', inplace=False)

In [293]:

main_df.columns.tolist()

['Age', 'Building', 'Cost', 'Effect', 'Heading']

In [155]:

main_df[main_columns_list].sample(5).T

,5,1,14,6,0
Heading,Cavalry,Cavalry,Economy technologies,Infantry,Building technologies
Technology,Scale Barding Armor,Husbandry,Coinage,Scale Mail Armor,Town Watch
Age,II,III,III,II,II
Building,Blacksmith,Stable,Market,Blacksmith,Town Center
Cost,150F,250F,200F 100G,100F,75F
Effect,Cavalry armor +1/+1,Cavalry speed +10%,Tribute fee 20%,Infantry armor +1/+1,Building LOS +4


In [7]:

def get_forg_technology_page(content_text_div, link_dict):
    for heading_span in content_text_div.select('span[class="mw-headline"]'):
        heading = heading_span.string.strip()
        if heading in ['General technologies', 'Unique technologies']:
            print('{}: {}'.format(heading_span.parent.name, heading))
            siblings_list = list(heading_span.parent.next_siblings)
            for sibling in siblings_list:
                tag = sibling.name
                if tag == 'ul':
                    for link in sibling.find_all('a'):
                        link_title = link['title']
                        link_dict[link_title] = link['href']
                    break
    
    return link_dict

In [170]:

forg_page_url = '{}/wiki/Age_of_Empires_II_HD:_The_Forgotten'.format(wiki_url)
forg_content_text_div = get_content_text_div(forg_page_url)
link_dict = get_forg_technology_page(forg_content_text_div, link_dict)

h4: General technologies
h4: Unique technologies


In [8]:

def get_afri_technology_page(content_text_div, link_dict):
    for heading_span in content_text_div.select('span[class="mw-headline"]'):
        heading = heading_span.string.strip()
        if heading in ['Technologies']:
            print('{}: {}'.format(heading_span.parent.name, heading))
            siblings_list = list(heading_span.parent.next_siblings)
            for sibling in siblings_list:
                tag = sibling.name
                if tag == 'ul':
                    for link in sibling.find_all('a'):
                        link_title = link['title']
                        link_dict[link_title] = link['href']
                    break
    
    return link_dict

In [175]:

afri_page_url = '{}/wiki/Age_of_Empires_II_HD:_The_African_Kingdoms'.format(wiki_url)
afri_content_text_div = get_content_text_div(afri_page_url)
link_dict = get_afri_technology_page(afri_content_text_div, link_dict)

h3: Technologies


In [9]:

space_regex = re.compile(r'\s+')
def process_small_list_of_units(appl_civ_text, row_dict):
    appl_civ_list = space_regex.split(appl_civ_text)
    for civ_name in civ_name_list:
        row_dict[civ_name] = 0
    for civ_name in appl_civ_list:
        if civ_name in civ_name_list:
            row_dict[civ_name] = 1
    
    return row_dict

In [10]:

def process_all_except(appl_civ_text, row_dict):
    for civ_name in civ_name_list:
        if civ_name in appl_civ_text:
            row_dict[civ_name] = 0
        else:
            row_dict[civ_name] = 1
    
    return row_dict

In [11]:

toc_selector = '#mw-content-text > table[class="toccolours"]'
def process_exceptions(link_soup, row_dict):
    desc_p_list = link_soup.select(toc_selector)
    if len(desc_p_list):
        for sib in desc_p_list[0].previous_siblings:
            if 'available to all civilizations except' in str(sib):
                break
        process_all_except(sib.text.strip(), row_dict)
    
    return row_dict

In [12]:

def process_article(art_table_list, row_dict):
    art_table = art_table_list[0]
    art_ul_list = art_table.find_all('ul')
    assert len(art_ul_list) == 2

    # Get available civs
    available_ul = art_ul_list[0]
    for link in available_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 1

    # Get unavailable civs
    unavailable_ul = art_ul_list[1]
    for link in unavailable_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 0

    return row_dict

In [13]:

article_selector = '#mw-content-text > table.article-table'
civ_name_list = load_object('civ_name_list')
def process_unit(link_soup, appl_civ_text, row_dict):
    for column_name in civ_name_list:
        columns_set.add(column_name)
    if 'All civilizations' in appl_civ_text:
        for civ_name in civ_name_list:
            row_dict[civ_name] = 1
    elif 'All except ' in appl_civ_text:
        row_dict = process_all_except(appl_civ_text, row_dict)
    elif 'See ' in appl_civ_text:
        art_table_list = link_soup.select(article_selector)
        if len(art_table_list):
            row_dict = process_article(art_table_list, row_dict)
        else:
            row_dict = process_exceptions(link_soup, row_dict)
    else:
        row_dict = process_small_list_of_units(appl_civ_text, row_dict)
    
    return row_dict

In [14]:

def consume_aside(unit_soup, row_dict):
    aside_list = unit_soup.select('aside')
    for aside_soup in aside_list:
        for data_source_div in aside_soup.find_all('div', {'data-source': True}):
            data_source_h3_list = data_source_div.find_all('h3')
            if len(data_source_h3_list):
                h3_siblings_list = list(data_source_h3_list[0].next_siblings)
                if len(h3_siblings_list):
                    column_name = data_source_div['data-source']
                    value_list = list(h3_siblings_list[1].strings)
                    column_value = ' '.join(x.strip() for x in value_list).strip()
                    if column_name == 'Available':
                        row_dict = process_unit(unit_soup, column_value, row_dict)
                    else:
                        columns_set.add(column_name)
                        row_dict[column_name] = column_value
    
    return row_dict

In [314]:

rows_list = []
index_list = []
columns_set = set(main_df.columns)
for technology_name, technology_url in link_dict.items():
    page_url = '{}{}'.format(wiki_url, technology_url)
    content_text_div = get_content_text_div(page_url)
    technology_name = technology_name.split(' (')[0]
    if technology_name not in index_list:
        index_list.append(technology_name)
        rows_list.append(consume_aside(content_text_div, {'technology_url': page_url}))

In [315]:

other_columns_set = columns_set - set(civ_name_list)
other_columns_set = other_columns_set - set(['Heading'])
columns_list = ['Heading'] + list(other_columns_set) + civ_name_list
technology_df = pd.DataFrame(data=rows_list, index=index_list,
                             columns=columns_list).dropna(axis='columns', how='all')

In [316]:

technology_df.sample(5).T.sample(5)

,Chatras,Bow Saw,Double Crossbow,Plate Mail Armor,Chivalry
Saracens,0,1,0,1,0
Next,NaN,Two-Man Saw,NaN,NaN,NaN
Khmer,0,1,1,0,0
Burmese,0,1,0,1,0
Teutons,0,1,0,1,0


In [317]:

technology_df.columns.tolist()

['Previous', 'Intro', 'Wood', 'Stone', 'Time', 'Required', 'Next', 'Gold', 'Building', 'Food', 'Age', 'Aztecs', 'Berbers', 'Britons', 'Burmese', 'Byzantines', 'Celts', 'Chinese', 'Ethiopians', 'Franks', 'Goths', 'Huns', 'Incas', 'Indians', 'Italians', 'Japanese', 'Khmer', 'Koreans', 'Magyars', 'Malay', 'Malians', 'Mayans', 'Mongols', 'Persians', 'Portuguese', 'Saracens', 'Slavs', 'Spanish', 'Teutons', 'Turks', 'Vietnamese', 'Vikings']

In [305]:

set.intersection(set(main_df.columns), set(technology_df.columns))

{'Building', 'Age'}

In [318]:

#main_df = main_df.set_index('Technology')
main_df.index.name = 'Technology'
len(main_df.index.tolist())

66

In [319]:

technology_df.index.name = 'Technology'
print(len(technology_df.index))
len(set.union(set(main_df.index), set(technology_df.index)))

135


135

In [320]:

technology_df = pd.merge(left=main_df, right=technology_df, how='right', on=['Building', 'Age'],
                         left_index=True, right_index=True, suffixes=('_main','_technology'))

In [321]:

file_path = os.path.join(DATA_CSV_FOLDER, 'technology_df.csv')
technology_df.to_csv(file_path)

In [324]:

save_dataframes(include_index=True, technology_df=technology_df)

Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\technology_df.csv


In [323]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'technology_df.xlsx')
technology_df.to_excel(file_path)

In [312]:

file_path = os.path.join(xlsx_folder, 'main_df.xlsx')
main_df.to_excel(file_path)